In [1]:
import requests
import dill
from bs4 import BeautifulSoup
from datetime import datetime
import simplejson as json
import pandas as pd
import numpy as np
from urllib.parse import quote
from urllib.request import urlopen
import spotipy
import time
import re

In [2]:
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = spotipy.oauth2.SpotifyClientCredentials('d6967ce2057448d4aab3ad9898119c97',  'ad7f82cc26a64f1595b6b3c4cd917243')
spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
def strip_Feat(artists):
    pos = artists.find('Featuring')
    if pos == -1:
        return artists
    else:
        return artists[:pos].strip()
    
def get_track_artists(year, domain = 'https://www.billboard.com/charts/year-end/'):
    path = domain + str(year) + '/hot-100-songs'
    response = requests.get(path)
    soup = BeautifulSoup(response.text, "lxml")

    parents = soup.findAll(class_=r'ye-chart-item__text')

    track_artists = []
    for parent in parents:
        track = parent.find('div',attrs={'class':'ye-chart-item__title'}).get_text().strip()
        artists = parent.find('div',attrs={'class':'ye-chart-item__artist'}).get_text().strip()
        track_artists.append((track, artists))
    return track_artists


def get_album_artists(year, domain='https://www.billboard.com/archive/charts/'):
    response = requests.get(domain + str(year) + "/top-album-sales")
    soup = BeautifulSoup(response.text, "lxml")
    table = soup.find('table',attrs={'class':'archive-table'}).findAll('tr')[1:]

    album_artists=[]
    for tr in table:
        td = tr.findAll('td')
        try:
            album_artists.append((td[-2].get_text(),td[-1].get_text()))
        except:
            pass
    return album_artists

def get_track_artists_arxiv(year, domain='https://www.billboard.com/archive/charts/'):
    path = domain + str(year) + '/hot-100'
    response = requests.get(path)
    soup = BeautifulSoup(response.text, "lxml")

    table = soup.find('table',attrs={'class':'archive-table'}).findAll('tr')[1:]

    track_artists=[]
    for tr in table:
        td = tr.findAll('td')
        try:
            track_artists.append((td[-2].get_text(),td[-1].get_text()))
        except:
            pass
    return track_artists

def get_album_id_from_track(track, artists):
    try:
        track = track.replace("'",'')
        artists = ", ".join(re.split(r' & | x | X | With | with ', artists))
        q = 'track:' + track + ' artist:' + artists
        result = spotify.search(q, limit=1)
        the_first_album = result['tracks']['items'][0]['album']
        album_id = the_first_album.get('id')
        return album_id
    except:
#         try:
#             artists = ' '.join(artists.split(' x '))
#             artists = ' '.join(artists.split(' X '))
#             q = track + ' ' + artists
#             result = spotify.search(q, limit=1)
#             the_first_album = result['tracks']['items'][0]['album']
#             album_id = the_first_album.get('id')
#             return album_id
#         except:
        return track, artists

def get_album_id_from_album(album, artists):

    try:
        album = album.replace("'",'')        
        artists = ", ".join(re.split(r' & | x | X | With | with ', artists))
        q = 'album:' + album + ' artist:' + artists
        result = spotify.search(q, type='album', limit=1)
        the_first_album = result['albums']['items'][0]
        album_id = the_first_album.get('id')
        return album_id
    
    except:
#         try:
#             q =  album
#             result = spotify.search(q, type='album', limit=1)
#             the_first_album = result['albums']['items'][0]
#             album_id = the_first_album.get('id')
#             return album_id
#         except:
        return album, artists

def get_albums_id_list(year, get_album_id_func, scraping_func):
    albums_id = []    


    trackOrAlbum_artists = scraping_func(year)
    for trackOrAlbum, artists in trackOrAlbum_artists:
        albums_id.append(get_album_id_func(trackOrAlbum,strip_Feat(artists)))
        
    return albums_id

def album_json(d):
    subkeys = ['id', 'name', 'genres',  'popularity', 'total_tracks']
    
    album_info = {k: d[k] for k in subkeys if k in d}
    tracks = d['tracks']['items']
    tracks_id = [t['id'] for t in tracks]
    tracks_df_json = pd.DataFrame(audio_features(tracks_id),columns=['id',  'track_number', 'popularity','name', 'duration_ms', 'tempo','time_signature', 'key',
       'valence', 'mode', 'acousticness', 'danceability', 'energy', 
       'instrumentalness',  'liveness', 'loudness', 'speechiness']).to_json(orient='split')
    album_info.update({'artists_list': [a['name'] for a in d['artists']], 'tracks_info': tracks_df_json })
    
    return album_info

def track_json(track_id):
    d = spotify.track(track_id)
    subkeys = ['id', 'popularity', 'name', 'track_number']
    subdict = {k: d[k] for k in subkeys if k in d}
    return subdict


def audio_feature_decorator(spotify_audio_features_func):

    def wrapper_func(track_id_list):
        subkeys = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']
        
        
        features_list = spotify_audio_features_func(track_id_list)
        
        my_features_list = []
        for i, features in enumerate(features_list):
            sub_features = {k: features[k] for k in subkeys if k in features}
            other_info = track_json(track_id_list[i])
            other_info.update(sub_features)
            
            my_features_list.append(other_info)
            
        return my_features_list
    
    return wrapper_func

@audio_feature_decorator
def audio_features(track_id_list):
    return spotify.audio_features(track_id_list)



In [4]:
def get_data(album_json):
    
    tracks_df = pd.read_json(album_json['tracks_info'], orient='split')

    return {k: album_json[k] for k in ['id', 'name', 'genres', 'popularity', 'total_tracks', 'artists_list']}, tracks_df
    

In [74]:
from sklearn.feature_extraction import DictVectorizer

In [744]:
# class Album(object):
#     def __init__(self, album_json):
        
#         self.id, self.name, self.genres, self.popularity, self.total_tracks, self.artists_list = [album_json[k] for k in ['id', 'name', 'genres', 'popularity', 'total_tracks', 'artists_list']]
        
#         self.tracks_df = pd.read_json(album_json['tracks_info'], orient='split')

#     def transform_features_(self):
        
#         cst = ColumnSelectTransformer(['acousticness', 'danceability', 'energy', 'liveness', 'speechiness','valence'])
#         poly = PolynomialFeatures(interaction_only=True, include_bias=False)
#         feat_matrix = poly.fit_transform(cst.transform(self.tracks_df))
#         min_max_scaler = MinMaxScaler()
        
#         return min_max_scaler.fit_transform(feat_matrix)
     
#     def transform_duration_tempo_(self):
        
#         return np.hstack((self.tracks_df[['duration_ms']].values/1e4, self.tracks_df[['tempo']]))

    
#     def transform_track_numbers_(self):
#         tnt = TrackNumberTransFormer()
#         return tnt.transform(self.tracks_df)
    
#     def transform_key_(self):
#         one_hot_key_matrix = np.zeros((self.total_tracks,12))
#         for i, k in enumerate(self.tracks_df['key']):
#             one_hot_key_matrix[i, k] = 1


        
#         return one_hot_key_matrix
        
#     def transform_labels_(self):
#         sorted_popularity = sorted(self.tracks_df['popularity'].values,reverse=True)
#         try:

#             the_first = sorted_popularity[0]
#             the_second = sorted_popularity[1]
#             the_last = sorted_popularity[-1]
#             shift_by = min(self.popularity, the_second)




#             shifted_popularity = self.tracks_df['popularity'].values - shift_by
#             transformed_popularity = np.zeros(self.total_tracks,dtype=np.float)

#     #         for i, t in enumerate(shifted_popularity):
#     #             if t < 0:

#     #                 transformed_popularity[i] =  t/ self.total_tracks
#     #             else:
#     #                 transformed_popularity[i] = t
#             for i, t in enumerate(shifted_popularity):
#                 if t < 0:

#                     transformed_popularity[i] =  - t / (the_last - shift_by)
#                 else:
#                     transformed_popularity[i] = t / (the_first - shift_by)

#             #print('before: ',sorted(self.tracks_df['popularity'].values,reverse=True))                
#             #print('after: ', sorted(transformed_popularity,reverse=True))        


#             return transformed_popularity
#         except:
#             popularity = self.popularity
#             popularity_list = self.tracks_df['popularity'].values
            
#             if len(sorted_popularity) == 2:
#                 l, s = sorted_popularity

#                 if l <= popularity:
#                     print(popularity_list - popularity,popularity,s)
#                     transformed_popularity = (popularity_list - popularity) / (popularity-s)
#                 elif s >= popularity:
#                     transformed_popularity = (popularity_list - popularity) / (l-popularity)


#                 else:
#                     norm_by = max(l-popularity, popularity-s)
#                     transformed_popularity = (popularity_list - popularity)/ norm_by

#             else:
#                 c = sorted_popularity[0]
#                 if c > popularity:
#                     transformed_popularity =  0.5 * np.ones(len(popularity_list))
#                 elif c < popularity: 
#                     transformed_popularity =  -0.5 * np.ones(len(popularity_list))
#                 else:
#                     transformed_popularity =  np.zeros(len(popularity_list))


#             return transformed_popularity

#     def transform(self):
#         return np.hstack((self.transform_track_numbers_(), self.transform_key_(), self.transform_duration_tempo_())), self.transform_features_(), self.transform_labels_()
                
        

In [75]:
def top_songs_albums_year(year, get_album_id_func, scraping_func):
    json_list = []
    wrong_queries = []
    time_out = []
    albums_id_list = get_albums_id_list(year, get_album_id_func, scraping_func)
    for album_id in albums_id_list:
        try:
            result = spotify.album(album_id)
            json_list.append(album_json(result))
        except:
            if type(album_id) == tuple:
                wrong_queries.append(album_id)
            else:
                try:
                    time.sleep(6)
                    result = spotify.album(album_id)
                    json_list.append(album_json(result))
                except:
                    time_out.append(album_id)
    fail = []
    for album_id in time_out:
        try:
            result = spotify.album(album_id)
            json_list.append(album_json(result))
            
        except:
            fail.append(album_id)

    
        
    return json_list, (year, fail, wrong_queries)
   

In [ ]:
[(2018,[("God's Plan", 'Drake'), ("Boo'd Up", 'Ella Mai'), ('Bodak Yellow (Money Moves)', 'Cardi B'), ("King's Dead", 'Jay Rock, Kendrick Lamar, Future, James Blake'), ('Te Bote', 'Casper Magico, Nio Garcia, Darell, Nicky Jam, Ozuna, Bad Bunny'), ('Lemon', 'N*E*R*D, Rihanna'), ("I'm Upset", 'Drake')]),
(2017, [("I'm The One", 'DJ Khaled'), ("There's Nothing Holdin' Me Back", 'Shawn Mendes'), ('Bodak Yellow (Money Moves)', 'Cardi B'), ("Can't Stop The Feeling!", 'Justin Timberlake'), ('Broccoli', 'D.R.A.M.')]),
 (2016,[("Don't Let Me Down", 'The Chainsmokers'), ("Can't Stop The Feeling!", 'Justin Timberlake'), ('Broccoli', 'D.R.A.M.'), ("Don't", 'Bryson Tiller'), ("Don't Mind", 'Kent Jones'), ("Ex's & Oh's", 'Elle King'), ("Can't Feel My Face", 'The Weeknd')]),
 (2015, [("Can't Feel My Face", 'The Weeknd'), ("I'm Not The Only One", 'Sam Smith'), ('G.D.F.R.', 'Flo Rida'), ('B**** Better Have My Money', 'Rihanna'), ("Ex's & Oh's", 'Elle King'), ('Tuesday', 'I LOVE MAKONNEN'), ("Don't", 'Ed Sheeran')]),
 (2014,[("Don't Tell 'Em", 'Jeremih'), ("Ain't It Fun", 'Paramore'), ("Don't", 'Ed Sheeran'), ("Can't Remember To Forget You", 'Shakira')]),
 (2013,[("Can't Hold Us", 'Macklemore, Ryan Lewis'), ('Holy Grail', 'Jay Z'), ("Don't You Worry Child", 'Swedish House Mafia'), ('Die Young', 'Ke$ha'), ("That's My Kind Of Night", 'Luke Bryan'), ("Mama's Broken Heart", 'Miranda Lambert'), ("I'm Different", '2 Chainz')]),
 (2012,[("Don't Wake Me Up", 'Chris Brown'), ("Let's Go", 'Calvin Harris'), ('Die Young', 'Ke$ha')]),
 (2011,[("What's My Name?", 'Rihanna'), ("I'm On One", 'DJ Khaled'), ("Don't You Wanna Stay", 'Jason Aldean, Kelly Clarkson'), ('Remind Me', 'Brad Paisley Duet, Carrie Underwood'), ("Don't Wanna Go Home", 'Jason Derulo')]),
 (2010,[('TiK ToK', 'Ke$ha'), ('Empire State Of Mind', 'Jay-Z + Alicia Keys'), ("Haven't Met You Yet", 'Michael Buble'), ('Young Forever', 'Jay-Z + Mr. Hudson'), ('Blah Blah Blah', 'Ke$ha'), ('Eenie Meenie', 'Sean Kingston, Justin Bieber')]),
 (2009,[("Don't Trust Me", '3OH!3'), ('Run This Town', 'Jay-Z, Rihanna, Kanye West'), ('Empire State Of Mind', 'Jay-Z + Alicia Keys'), ('Turn My Swag On', "Soulja Boy Tell'em"), ('People Are Crazy', 'Billy Currington')]),
 (2008,[("Don't Stop The Music", 'Rihanna'), ("Can't Believe It", 'T-Pain'), ("It's Not My Time", '3 Doors Down'), ('Crank That (Soulja Boy)', "Soulja Boy Tell'em"), ("Can't Help But Wait", 'Trey Songz'), ("You're Gonna Miss This", 'Trace Adkins')]),
 (2007,[("Don't Matter", 'Akon'), ("It's Not Over", 'Daughtry'), ('Crank That (Soulja Boy)', "Soulja Boy Tell'em"), ("Cupid's Chokehold/Breakfast In America", 'Gym Class Heroes'), ("I'm A Flirt", 'R. Kelly Or Bow Wow ('), ('Go Getta', 'Young Jeezy'), ('Same Girl', 'R. Kelly Duet, Usher')]),
 (2006,[("You're Beautiful", 'James Blunt'), ("It's Goin' Down", 'Yung Joc'), ('Snap Yo Fingers', 'Lil Jon'), ("Ain't No Other Man", 'Christina Aguilera'), ("I'm N Luv (Wit A Stripper)", 'T-Pain'), ("Where'd You Go", 'Fort Minor'), ("Don't Forget About Us", 'Mariah Carey'), ("What's Left Of Me", 'Nick Lachey'), ('Soul Survivor', 'Young Jeezy'), ("I'm Sprung", 'T-Pain')])]

In [48]:
fail_list

[(2018,
  [],
  [('M A N I A', 'Fall Out Boy'),
   ('My Dear Melancholy, (EP)', 'The Weeknd'),
   ('Mamma Mia! Here We Go Again', 'Soundtrack')]),
 (2017,
  [],
  [('Moana', 'Soundtrack'),
   ('The Search For Everything : Wave One (EP)', 'John Mayer'),
   ('4:44', 'JAY-Z')]),
 (2016,
  [],
  [('Lemonade', 'Beyonce'),
   ('Lemonade', 'Beyonce'),
   ('Suicide Squad: The Album', 'Soundtrack')]),
 (2015,
  [],
  [('NOW 53', 'Various Artists'),
   ('Pitch Perfect 2', 'Soundtrack'),
   ('Descendants', 'Soundtrack')]),
 (2014,
  [],
  [('Frozen', 'Soundtrack'),
   ('Frozen', 'Soundtrack'),
   ('NOW 49', 'Various Artists'),
   ('Frozen', 'Soundtrack'),
   ('Frozen', 'Soundtrack'),
   ('Guardians Of The Galaxy: Awesome Mix Vol. 1', 'Soundtrack')]),
 (2013,
  [],
  [('Les Miserables', 'Soundtrack'),
   ('Magna Carta... Holy Grail', 'JAY-Z'),
   ('The Marshall Mathers LP 2', 'Eminem'),
   ('The Marshall Mathers LP 2', 'Eminem'),
   ('Blame It All On My Roots: Five Decades Of Influences', 'Garth B

In [ ]:
fail_list_2 = []
for year in reversed(range(2000,2006)):
    data, fail = top_songs_albums_year(year,get_album_id_from_track,get_track_artists_arxiv)
    fail_list_2.append(fail)
    with open('top_songs_albums_v3_year_{}'.format(year),'w') as outfile:
        json.dump(data,outfile)


In [47]:
fail_list = []
for year in reversed(range(2000,2019)):
    data, fail = top_songs_albums_year(year,get_album_id_from_album)
    fail_list.append(fail)
    with open('best_seller_albums_v3_year_{}'.format(year),'w') as outfile:
        json.dump(data,outfile)

retrying ...4secs
retrying ...1secs
retrying ...3secs
retrying ...1secs
retrying ...2secs
retrying ...1secs
retrying ...2secs
retrying ...1secs
retrying ...4secs
retrying ...1secs
retrying ...3secs
retrying ...1secs
retrying ...3secs
retrying ...1secs
retrying ...4secs
retrying ...1secs
retrying ...3secs
retrying ...1secs
retrying ...3secs
retrying ...1secs
retrying ...3secs
retrying ...1secs
retrying ...2secs
retrying ...1secs
retrying ...3secs
retrying ...1secs
retrying ...2secs
retrying ...1secs
retrying ...3secs
retrying ...1secs
retrying ...4secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...3secs
retrying ...1secs
retrying ...4secs
retrying ...1secs
retrying ...3secs
retrying ...1secs
retrying ...2secs
retrying ...1secs
retrying ...3secs
retrying ...1secs
retrying ...3secs
retrying ...1secs
retrying ...3secs
retrying ...1secs
retrying ...2secs
retrying ...1secs
retrying ...3secs
retrying ...1secs
retrying ...4secs
retrying ...1secs
retrying ...2secs
retrying .

In [9]:
for year in reversed(range(2006,2019)):
    data = top_songs_albums_year(year,get_album_id_from_track)
    with open('top_songs_albums_v3_year_{}'.format(year),'w') as outfile:
        json.dump(data,outfile)

retrying ...1secs
retrying ...1secs
retrying ...3secs
retrying ...1secs
retrying ...3secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...3secs
retrying ...1secs
retrying ...3secs
retrying ...1secs
retrying ...4secs
retrying ...1secs
retrying ...3secs
retrying ...1secs
retrying ...4secs
retrying ...1secs
retrying ...3secs
retrying ...1secs
2018 [] [("God's Plan", 'Drake'), ("Boo'd Up", 'Ella Mai'), ('Bodak Yellow (Money Moves)', 'Cardi B'), ("King's Dead", 'Jay Rock, Kendrick Lamar, Future, James Blake'), ('Te Bote', 'Casper Magico, Nio Garcia, Darell, Nicky Jam, Ozuna, Bad Bunny'), ('Lemon', 'N*E*R*D, Rihanna'), ("I'm Upset", 'Drake')]
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...4secs
retrying ...1secs
retrying ...3secs
retrying ...1secs
retrying ...3secs
retrying ...4secs
retrying ...1secs
retrying ...3secs
retrying ...1secs
retrying ...3secs
retrying ...1secs
retrying ...3secs
retrying ...1secs
retrying ...3secs
retrying ...1

In [8]:
def load_files(year):

    with open("best_seller_albums_v3_year_" + str(year)) as json_file:  
        best_seller_albums_data = json.load(json_file)
        best_seller_albums_albums = [Album(d) for d in best_seller_albums_data]

    with open("top_songs_albums_v3_year_" + str(year)) as json_file:  
        top_songs_albums_data = json.load(json_file)
        top_songs_albums_albums = [Album(d) for d in top_songs_albums_data]

    return best_seller_albums_albums, top_songs_albums_albums

In [64]:
a = album_list[2]
a.artists_list

['Panic! At The Disco']

In [5]:
def print_artists(ls):
    if len(ls) == 2:
        return ' & '.join(ls)
    else:
        return ', '.join(ls[:-1]) + ' & ' + ls[-1]

In [69]:
print_artists(['Alice','Bob', 'Cat'])

'Alice, Bob & Cat'

In [6]:
class Album(object):
    def __init__(self, album_json):
        self.id, self.name, self.genres, self.popularity, self.total_tracks, self.artists_list = [album_json[k] for k in ['id', 'name', 'genres', 'popularity', 'total_tracks', 'artists_list']]
        self.tracks_df = pd.read_json(album_json['tracks_info'], orient='split')

    def unit_transf(self):
        self.tracks_df['tempo'] = self.tracks_df['tempo'] / 60
        self.tracks_df['duration_ms'] = self.tracks_df['duration_ms'] / 1000 / 60
        self.tracks_df['loudness'] = self.tracks_df['loudness'] / 10
        self.tracks_df['ordering'] = MinMaxScaler().fit_transform(self.tracks_df['track_number'].values.reshape(-1,1))
        

    def classification_label(self):
        self.tracks_df['label'] traself.tracks_df['popularity'] >= self.popularity
        
    def train_X_y(self):
        X = self.tracks_df[['mode', 'tempo', 'duration_ms', 'ordering', 'acousticness', 'danceability', 'energy', 'liveness', 'speechiness','valence']].values
        y = self.tracks_df['label'].values
        return X, y
    
    
    def transform_features_(self):
        
        cst = ColumnSelectTransformer(['acousticness', 'danceability', 'energy', 'liveness', 'speechiness','valence'])
        poly = PolynomialFeatures(interaction_only=False, include_bias=True)
        feat_matrix = poly.fit_transform(cst.transform(self.tracks_df))
        return feat_matrix
        min_max_scaler = MinMaxScaler()
        
        return min_max_scaler.fit_transform(feat_matrix)
     
    def transform_duration_tempo_(self):
        
        return np.hstack((self.tracks_df[['duration_ms']].values/ 1000 / 60, self.tracks_df[['tempo']]))

    
    def transform_track_numbers_(self):
        tnt = TrackNumberTransFormer()
        return tnt.transform(self.tracks_df)
    
    def transform_key_(self):
        one_hot_key_matrix = np.zeros((self.total_tracks,12))
        for i, k in enumerate(self.tracks_df['key']):
            one_hot_key_matrix[i, k] = 1


        
        return one_hot_key_matrix
        
    def transform_labels_(self):
        return self.tracks_df['popularity'].values
        
        sorted_popularity = sorted(np.unique(self.tracks_df['popularity'].values),reverse=True)
        try:
            the_first = sorted_popularity[0]
            the_second = sorted_popularity[1]
            the_last = sorted_popularity[-1]
            shift_by = min(self.popularity, the_second)




            shifted_popularity = self.tracks_df['popularity'].values - shift_by
            transformed_popularity = np.zeros(self.total_tracks,dtype=np.float)

    #         for i, t in enumerate(shifted_popularity):
    #             if t < 0:

    #                 transformed_popularity[i] =  t/ self.total_tracks
    #             else:
    #                 transformed_popularity[i] = t



            for i, t in enumerate(shifted_popularity):
                if t < 0:

                    transformed_popularity[i] =  - t / (the_last - shift_by)


                else:
                    transformed_popularity[i] = t / (the_first - shift_by)

            #print('before: ',sorted(self.tracks_df['popularity'].values,reverse=True))                
            #print('after: ', sorted(transformed_popularity,reverse=True))        


            return transformed_popularity
        except:
            if len(sorted_popularity) == 2:
                l, s = sorted_popularity

                if l <= self.popularity:
                    transformed_popularity = (self.tracks_df['popularity'].values - self.popularity) / (self.popularity-s)
                elif s >= self.popularity:
                    transformed_popularity = (self.tracks_df['popularity'].values - self.popularity) / (l-self.popularity)
                    
                
                else:
                    norm_by = max(l-self.popularity, self.popularity-s)
                    transformed_popularity = (self.tracks_df['popularity'].values - self.popularity)/ norm_by
                    
                    
                
            else:
                c = sorted_popularity[0]
                if c > self.popularity:
                    transformed_popularity =  0.5 * np.ones(self.total_tracks)
                elif c < self.popularity: 
                    transformed_popularity =  -0.5 * np.ones(self.total_tracks)
                else:
                    transformed_popularity =  np.zeros(self.total_tracks)
            return transformed_popularity
                    

    def transform(self):
        return np.hstack((self.transform_track_numbers_(), self.transform_key_(), self.transform_duration_tempo_())), self.transform_features_(), self.transform_labels_()
                
        

In [9]:
album_list, song_list = load_files(2014)
a = (album_list + song_list)[51]

In [18]:
model = dill.load(open('model.pkd', 'rb'))


In [772]:
a.transform_track_numbers_().shape, a.transform_key_().shape, a.transform_duration_tempo_().shape, a.transform_features_().shape

((50, 3), (59, 12), (50, 2), (50, 21))

In [773]:
a.total_tracks

59

In [769]:
a.id

'7lZs5r4oQV2nutddffLrg0'

In [709]:
for i, a in enumerate((album_list+song_list)):
    if np.isnan(a.transform_labels_()).any():
        print(i)

In [710]:
a.transform_labels_()

array([[-0.5       ],
       [-0.8       ],
       [-0.1       ],
       [ 1.        ],
       [-0.6       ],
       [-0.7       ],
       [-0.9       ],
       [-0.95      ],
       [-0.9       ],
       [ 0.66666667],
       [-0.95      ],
       [-1.        ]])

In [5]:
album_list, song_list = load_files(2018)

In [7]:
a = album_list[0]
a.transform()

NameError: name 'TrackNumberTransFormer' is not defined

In [9]:
from sklearn import base
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler


In [10]:
class ColumnSelectTransformer(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self, col_names):
        self.col_names = col_names  # We will need these in transform()
    
    def fit(self, X, y=None):
        # This transformer doesn't need to learn anything about the data,
        # so it can just return self without any further processing
        return self
    
    def transform(self, X):
        df = pd.DataFrame(X)
        return df[self.col_names].values.tolist()
        # Return an array with the same number of rows as X and one
        # column for each in self.col_names

In [243]:

cst = ColumnSelectTransformer(['acousticness', 'danceability', 'energy', 'liveness', 'speechiness','valence'])
poly = PolynomialFeatures(interaction_only=True, include_bias=False)
feat_matrix = poly.fit_transform(cst.transform(a.tracks_df))

min_max_scaler = MinMaxScaler()
min_max_scaler.fit_transform(feat_matrix)[:,2]

array([0.58576052, 0.53559871, 0.55825243, 0.7605178 , 0.72977346,
       0.69579288, 0.58899676, 0.5420712 , 0.96601942, 0.82847896,
       1.        , 0.        ])

In [11]:
from sklearn.preprocessing import OneHotEncoder

In [12]:
class KeyModeTransFormer(base.BaseEstimator, base.TransformerMixin):
    
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        df = pd.DataFrame(X)
        key_mode = df[['key',  'mode']].values
            
        enc = OneHotEncoder(handle_unknown='ignore')
        
        
        return enc.fit_transform(key_mode)
        # Return an array with the same number of rows as X and one
        # column for each in self.col_names

In [13]:
class TrackNumberTransFormer(base.BaseEstimator, base.TransformerMixin):
    
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        df = pd.DataFrame(X)
        track_index = df.index.values
        ar = np.arange(len(track_index)) 
        g = np.array_split(ar,3)
        
        d = {}
        for i in g[0]:
            d[i] = 1
        for i in g[1]:
            d[i] = 2
        for i in g[2]:
            d[i] = 3
            

        into_three = np.array([d[i] for i in track_index]).reshape(-1,1)

        
        enc = OneHotEncoder(sparse=False,handle_unknown='ignore')
        enc.fit_transform(into_three)
        
        return enc.fit_transform(into_three)
        # Return an array with the same number of rows as X and one
        # column for each in self.col_names

In [32]:
def X_y(album_list, dim_l, dim_n):
    
    X_n, X_l = np.empty((0,dim_n)), np.empty((0,dim_l))
    y = []

    for i, a in enumerate(album_list):

        if a.total_tracks >=3 and a.total_tracks <= 50:
            a_X_n, a_X_l, a_y = a.transform()
            X_l = np.vstack(( a_X_l,X_l))
            X_n = np.vstack((X_n, a_X_n))
            
            y = np.hstack((y, a_y))
#         else:
#             print(i,a.total_tracks,len(a.tracks_df))
    return X_l, X_n, y

In [33]:
ls = []
for year in [2018,2016]:
    print(year)
    album_list, song_list = load_files(year)
    ls += (album_list + song_list)



X_l, X_n, y = X_y(ls,28,17)

2018
2016


In [25]:
len(X_l)

3446

In [602]:
from sklearn import model_selection
from sklearn import metrics
cv = model_selection.KFold(n_splits=3, random_state=0, shuffle=True)


In [598]:
class EstimatorTransformer(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self, estimator):
        self.estimator = estimator # What needs to be done here?
        
    def fit(self, X, y):
        #self.estimator.fit(X, y) # Fit the stored estimator.
        return self # Question: what should be returned?
    
    def transform(self, X):
        y_predict = self.estimator.predict(X) # Use predict on the stored estimator as a "transformation".
        return np.array(y_predict).reshape(-1,1) # Be sure to return a 2-D array.

In [597]:
from sklearn.pipeline import FeatureUnion

union = FeatureUnion([
        ('nonlinear', EstimatorTransformer(nonLin_est)),
        ('linear', EstimatorTransformer(Lin_est))
    
    ])

NameError: name 'EstimatorTransformer' is not defined

In [661]:
np.isfinite(y).all()
np.isnan(y).any()
indices = np.arange(len(y)).reshape(-1, 1)
indices[np.isnan(y)]

array([  34,   41,   43,   50,   78,   80,   92,   99,  405,  408,  845,
        846,  980,  982,  985, 1070, 1075, 1320, 1334, 1583, 1584, 1600,
       1614, 1653, 1654])

In [728]:
Lin.best_params_

{'alpha': 100}

In [730]:
nonLin.best_params_

{'n_estimators': 150}

In [731]:
y

array([[ 0.        ],
       [-0.01666667],
       [-0.2       ],
       ...,
       [-1.        ],
       [-0.8125    ],
       [-0.5625    ]])

In [793]:
Lin.best_params_

{'alpha': 300}

In [832]:
class ResidualEstimator(base.BaseEstimator, base.RegressorMixin):
    
    def __init__(self, linReg, nonLinReg, alpha, max_depth):
        self.linReg = linReg
        self.alpha = alpha
        self.nonLinReg = nonLinReg
        self.max_depth = max_depth
        
    def fit(self, X, y):
        self.linReg.set_params(alpha=self.alpha)
        self.nonLinReg.set_params(max_depth=self.max_depth)
        self.linReg.fit(X, y)
        
        residual_part = y - self.linReg.predict(X)

        self.nonLinReg.fit(X, residual_part)
    
        return self
        
    def predict(self, X):
        
        linear_part = self.linReg.predict(X)
        non_linear_part = self.nonLinReg.predict(X)
        return linear_part + non_linear_part
        

In [14]:
album_list, song_list = load_files(2018)

In [21]:
a = album_list[0]
a.transform()[0].shape,a.transform()[1].shape

((19, 17), (19, 28))

In [909]:
nn = 2
X_l_test, X_n_test, y_test = X_y([album_list[nn]])
s = gbr.predict(X_l_test)
sorted(range(len(s)), key=lambda k: s[k],reverse=True)

[4, 7, 6, 5, 3, 8, 2, 0, 1]

In [910]:
a = album_list[nn]
a.tracks_df.sort_values('popularity',ascending=False).index.values

array([0, 7, 1, 2, 3, 5, 6, 4, 8])

In [35]:
gbr.best_params_, gbr.best_score_

({'alpha': 0.1, 'max_depth': 5}, -0.17322932854962417)

In [34]:
gbr = GridSearchCV(GradientBoostingRegressor(random_state=0),{"alpha":np.arange(0.1,1,0.2),"max_depth":[5,10,15]})
gbr.fit(X_l,y)

GridSearchCV(cv=None, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=0,
             subsample=1.0, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'alpha': array([0.1, 0.3, 0.5, 0.7, 0.9]), 'max_depth': [5, 10, 15]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [26]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge

In [849]:


linReg = Ridge(alpha=10)

nonLinReg = RandomForestRegressor(random_state=10,n_estimators=150)

re = GridSearchCV(ResidualEstimator(linReg, nonLinReg, 10, 20),{"alpha":[30,50,70], "max_depth":[20,50,70]})
re.fit(X_l,y)


  # run each hyperparameter in one of two parallel jobs,scoring='neg_mean_squared_error'

KeyboardInterrupt: 

In [848]:
re.best_params_

{'alpha': 70, 'max_depth': 20}

In [ ]:
RandomForestRegressor(random_state=0,max_depth=10,n_estimators=150)

In [593]:
class AlbumPridModel(base.BaseEstimator, base.TransformerMixin):
    def __init__(self):
        pass

    def transform_(self, y, total):
        pass
        
    def transform(self, X):
        pass

    def fit_transform(self,X, y=None):
        pass
        
    

In [ ]:
from sklearn.pipeline import Pipeline
